# developing and testing pynetstim

In [1]:
import nibabel as nib
import numpy as np

from surfer import Brain
from mayavi import mlab
%gui qt
from utils import Coords,Surf, FreesurferSurf, FreesurferCoords
from points import Targets
from project import StimProject
import matplotlib.pyplot as plt
%matplotlib inline
#import seaborn as sns
#sns.set_context('notebook')
#sns.set_style('ticks')
from pymisc.plotting import clean_plot
from collections import defaultdict
from plotting import plot_points

**The aim is to add these features:**

- visualization of brain surfaces 
- adding stimulation sites to the brain surface for visualization 
- creating stimulation ROIs based on the stimulation site (ROI class maybe?)

In [2]:
subject = 'example_freesurfer_subject'
freesurfer_dir = '/Users/stadayon/Dropbox/projects/broad/projects/pynetstim/pynetstim/examples/example_data/'
project_dir = '/Users/stadayon/Dropbox/projects/broad/projects/pynetstim/pynetstim/temp'
brainsight_file = './examples/example_data/brainsightsession_example_data.txt'
project = StimProject(subject,freesurfer_dir,project_dir,brainsight_file)
project.summary()

In [3]:
targets = project.brainsight_targets.get_coords()
targets_obj = Targets(targets, subject,freesurfer_dir)
targets_obj.create_surf_rois(10,annot='Yeo2011_7Networks_N1000')
samples_df = project.brainsight_samples.get_samples('Session 1')
samples = samples_df[samples_df.dist_to_target<60][['loc_x','loc_y','loc_z']]
samples_obj = FreesurferCoords(samples,subject, freesurfer_dir)
#targets_obj.plot_points(hemi = 'both',show_rois=False,surf='white', annot='aparc',map_to_annot='aparc', scale_factor=.3, show_names=False,opacity=1,skin=False,map_surface='white')
#project.plot_points(samples_obj,surf='white',hemi='both',annot='aparc',scale_factor=4,
                    #img_basename='stims',background='white',opacity=.3)

In [4]:
plot_points(targets_obj,'both','pial','pial',map_to_annot='Yeo2011_7Networks_N1000',show_rois=True,show_names=True,scale_factor=.5)

In [4]:
plot_points?